# Prediction 2 
Using python

In [48]:
import pandas as pd
import numpy as np
import xgboost as xgb
from haversine import haversine


In [2]:
train_dataset_raw = pd.read_csv('data/train.csv')
train_dataset_raw.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


Reduce the dataset to be much smaller for working purposes

In [12]:
N = 1000

train_dataset = train_dataset_raw.sample(N)

## Filter errornous values

In [13]:
train_dataset = train_dataset_raw

# Trips too long
MAX_TRIP_DURATION = 2 * 3600
MIN_TRIP_DURATION = 30
train_dataset = train_dataset_raw[
    (train_dataset.trip_duration < MAX_TRIP_DURATION) &
    (train_dataset.trip_duration > MIN_TRIP_DURATION)
]

## Create new features

In [56]:
# Harvesine Distance

def create_distance(df):
    df['havers_dist'] = df.apply(lambda x: haversine(
        (x['pickup_latitude'], x['pickup_longitude']),
        (x['dropoff_latitude'], x['dropoff_latitude'])), axis=1)

create_distance(train_data)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
def create_datetime(df):

    df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])

    # Hour of the day
    df['pickup_seconds'] = (
        df['pickup_datetime'].dt.second +
        df['pickup_datetime'].dt.minute * 60 +
        df['pickup_datetime'].dt.hour * 60 * 60
    )

    # Day and month
    df['pickup_weekday'] = df['pickup_datetime'].dt.dayofweek
    df['pickup_month'] = df['pickup_datetime'].dt.month


def create_log_trip_duration(df):
    # Log trip_duration
    df['log_trip_duration'] = np.log(df["trip_duration"])

create_datetime(train_dataset)
create_log_trip_duration(train_dataset)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [41]:
train_dataset.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,havers_dist,pickup_seconds,pickup_weekday,pickup_month,log_trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,8813.951036,62695,0,3,6.120297
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,8816.882210,2615,6,6,6.496775
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,8815.215586,41724,1,1,7.661056
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,8820.602585,70351,2,4,6.061457
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,8811.061268,48655,5,3,6.075346


In [44]:
len(train_dataset['log_trip_duration'])

1451501

# Filter erronous values (after feature engineering)

In [115]:
msk = np.random.rand(len(train_dataset)) < 0.9

test_set = train_dataset[msk]
train_set = train_dataset[~msk] 

In [116]:
train_data = train_set[[
        'vendor_id',
        'passenger_count',
        'pickup_longitude',
        'pickup_latitude',
        'dropoff_longitude',
        'dropoff_latitude',
        # 'trip_duration',
        'havers_dist',
        'pickup_seconds',
        'pickup_weekday',
        'pickup_month',
        # 'log_trip_duration',
    ]]
train_label = train_set['log_trip_duration']
dtrain = xgb.DMatrix(train_data, label=train_label)

test_data = test_set[[
        'vendor_id',
        'passenger_count',
        'pickup_longitude',
        'pickup_latitude',
        'dropoff_longitude',
        'dropoff_latitude',
        # 'trip_duration',
        'havers_dist',
        'pickup_seconds',
        'pickup_weekday',
        'pickup_month',
        # 'log_trip_duration',
    ]]
test_label = test_set['log_trip_duration']
dtest = xgb.DMatrix(test_data, label=test_label)

In [117]:
param = {
    'lambda': .1,
    'alpha': .1,
    'feature_selector': 'shuffle',
    'objective': 'reg:linear',
    'nthread': 4
}
evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 100

bst = xgb.train(param, dtrain, num_round, evallist)

[16:20:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=6
[0]	eval-rmse:4.23506	train-rmse:4.23349
[16:20:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[1]	eval-rmse:2.99612	train-rmse:2.99465
[16:20:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[2]	eval-rmse:2.13951	train-rmse:2.13817
[16:20:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[3]	eval-rmse:1.55317	train-rmse:1.55182
[16:20:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[4]	eval-rmse:1.15715	train-rmse:1.15568
[16:20:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[5]	eval-rmse:0.898867	train-rmse:0.89709
[16:20:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 ext

## Prediction

In [118]:
test_dataset_raw = pd.read_csv('data/test.csv')

In [119]:
test_dataset = test_dataset_raw

create_distance(test_dataset)
create_datetime(test_dataset)

In [120]:
dtest = xgb.DMatrix(test_dataset[[
        'vendor_id',
        'passenger_count',
        'pickup_longitude',
        'pickup_latitude',
        'dropoff_longitude',
        'dropoff_latitude',
        # 'trip_duration',
        'havers_dist',
        'pickup_seconds',
        'pickup_weekday',
        'pickup_month',
        # 'log_trip_duration',
    ]])
ypred = bst.predict(dtest)

In [121]:
prediction_result = np.round(np.exp(ypred))

id0621643
1265.0


In [122]:
prediction_df = test_dataset
prediction_df['trip_duration'] = prediction_result

prediction_df.to_csv("submition2.csv", columns=['id', 'trip_duration'], index=False)